In [1]:
import os
import glob
import csv #Thao tac voi he thong tep
from PIL import Image #Xu ly anh(mo, chuyen doi dinh dang)
from pillow_heif import register_heif_opener #mo dinh dang .heic
register_heif_opener()

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models


In [ ]:
class MyDigitDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png', '*.heic', '*.jfif'):
            # Quét đệ quy tất cả ảnh trong các thư mục con
            self.image_paths.extend(glob.glob(os.path.join(root_dir, '**', ext), recursive=True))

        # Kiểm tra và lọc các ảnh hợp lệ
        valid_paths = []
        for path in self.image_paths:
            try:
                Image.open(path).verify()
                valid_paths.append(path)
            except:
                print(f"Ảnh lỗi (bỏ qua): {path}")
        self.image_paths = valid_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            label = int(os.path.basename(img_path).split('_')[0])
            if self.transform:
                image = self.transform(image)
            return image, label
        except:
            print(f"Lỗi ảnh: {img_path}")
            return None
# Quét tất cả ảnh trong thư mục training với các định dạng được hỗ trợ
# Label được trích xuất từ tên file (ký tự đầu tiên trước dấu '_')

# Ví dụ: 5_image001.jpg → label = 5


# Convert ảnh sang RGB để đảm bảo 3 channels
# Áp dụng transform (data augmentation) nếu có
# Xử lý lỗi khi ảnh bị hỏng
class MyTestDataset(Dataset): #de quy het tap test
    def __init__(self, folder, transform=None):
        self.transform = transform
        self.image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png', '*.heic', '*.jfif'):
            self.image_paths.extend(glob.glob(os.path.join(folder, '**', ext), recursive=True))

        valid_paths = []
        for path in self.image_paths:
            try:
                Image.open(path).verify()
                valid_paths.append(path)
            except:
                print(f"Ảnh lỗi (bỏ qua): {path}")
        self.image_paths = valid_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path
# Tìm kiếm đệ quy trong các thư mục con (recursive=True)
# Kiểm tra tính hợp lệ của ảnh trước khi thêm vào danh sách
# Trả về đường dẫn file thay vì label (vì test set không có label)

NameError: name 'Dataset' is not defined

In [ ]:
transformtrain = transforms.Compose([
    transforms.Resize((224, 224)), #resize ve 224x224
    transforms.RandomHorizontalFlip(), #dao chieu random de nhan dien chinh xac hon
    transforms.ToTensor(), #chuyen anh sang tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transformtest = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], #gia tri trung binh 3 mau rgb
                         std=[0.229, 0.224, 0.225]) #do lech chuan 3 mau rgb
])


In [4]:
root_path = r'D:\CS114_P21\Doan\train'
test_folder = r'D:\CS114_P21\Doan\test'
traindata = MyDigitDataset(root_path, transform=transformtrain)
traindata = [item for item in traindata if item is not None]
train_loader = DataLoader(traindata, batch_size=64, shuffle=True, num_workers=0)

testdata = MyTestDataset(test_folder, transform=transformtest)
test_loader = DataLoader(testdata, batch_size=64, shuffle=False, num_workers=0)

print("Train samples:", len(traindata))
print("Test samples:", len(testdata))
# batch_size=64: Xử lý 64 ảnh cùng lúc
# shuffle=True cho training: Trộn ngẫu nhiên dữ liệu
# shuffle=False cho test: Giữ thứ tự để map kết quả với file
# num_workers=0: Không dùng multiprocessing (tránh lỗi trên Windows)

Lỗi ảnh: D:\CS114_P21\Doan\train\gốc 9.jpg
Lỗi ảnh: D:\CS114_P21\Doan\train\img_17.jpg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\2c18ee0e7cea8354149df435532d74ae.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\3a816aa78f56749a0822d700ff560924.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\4abc1b5dcf1be1de6503dc072e132fa0.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\70ebf0bec317006017a54d6c9172af45.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\8eeee227b3f244e980b747387bc79bf2.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\9e748b9617e26b90011f8d7c3f8a7eee.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\a008f795ae8498751f2e5feeb0b73387.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\ce2388179ec73203a60d8efedd04e0f1.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\d4b080c6ce4933f1199fad7f75e7d112.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\e21dcb1cb0571a21a013b4232d1b6a13.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\f7679411e290bfd8dc90b3eadb57460a.jpeg
Ảnh lỗi (bỏ qua): D:\CS114_P21\Doan\test\ffd

In [5]:
#Khoi tao mo hinh
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# nn.CrossEntropyLoss():
# tính xác xuất softmax : từng kết quả chưa chuẩn hóa chia cho tổng các kết quả chưa chuẩn hóa


d:\Python3_10\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Python3_10\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
#Danh gia do chinh xac
def evaluate(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total if total > 0 else 0


In [ ]:
#Qua trinh huan luyen
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_acc = evaluate(model, train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {train_acc*100:.2f}%")
# Forward pass: Tính output từ input
# Loss calculation: So sánh output với ground truth
# Backward pass: Tính gradient
# Parameter update: Cập nhật weights
# Evaluation: Tính accuracy trên tập training


In [ ]:
#Du doan va xuat ket qua
model.eval()
results = []
with torch.no_grad():
    for images, img_paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        preds = preds.cpu().numpy()
        for path, pred in zip(img_paths, preds):
            filename = os.path.basename(path)
            results.append([filename, int(pred)])

with open('WECODE_RESNET18.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'prediction'])
    writer.writerows(results)
# Chuyển model sang evaluation mode
# Dự đoán trên từng batch của test set
# Map mỗi file với prediction tương ứng
# Xuất kết quả ra file CSV với format: filename, prediction